In [ ]:
# import packages 
import numpy as np
import time
import cvxpy as cp

In [ ]:
# this notebook implements the convex NN for the two-layer "fully connected" architecture
# for solving the convex problem, this notebook uses the 'cvxpy' package

In [ ]:
# paper link: https://arxiv.org/pdf/2101.02429.pdf

### Functions

In [ ]:
# poly act scalar case
# main function
def convexNN_poly_act_solver(A, y, beta, scs_max_iters, loss_type, a, b, c):
    d = A.shape[1]
    X_V = compute_X_V_base_FC(A, d, verbose)
    X_KTX_K, X_KTy, y_normsq, X_V_scaled = scale_X_K(X_V, y, a, b, c, d)
    Z, Z_prime, noncvx_cost = solve_cvx(X_KTX_K, X_KTy, y_normsq, d, beta, scs_max_iters, loss_type, X_V_scaled, y)
    return Z, Z_prime, noncvx_cost

# helper functions
def compute_X_V_base_FC(A, d, verbose=False):
    n = A.shape[0]

    X_V = np.zeros((n, d**2+d+1))
    for i in range(n):
        if i % 100 == 0 and verbose:
            print(i, end=", ")
        
        x_i = A[i:i+1,:].T
        X_V[i, 0:d**2] = np.matmul(x_i, x_i.T).reshape((d**2))
        X_V[i, d**2:d**2+d] = x_i.reshape((d))
        X_V[i, d**2+d] = 1
    return X_V
def scale_X_K(X_K, y, a, b, c, ff): # scale, here, refers to multiplying the columns by a,b,c
    X_K_scaled = X_K.copy()
    X_K_scaled[:, 0:ff**2] = a * X_K_scaled[:, 0:ff**2]
    X_K_scaled[:, ff**2:ff**2+ff] = b * X_K_scaled[:, ff**2:ff**2+ff]
    X_K_scaled[:, ff**2+ff] = c
    
    X_KTX_K = np.matmul(X_K_scaled.T, X_K_scaled)
    X_KTy = np.matmul(X_K_scaled.T, y)
    y_normsq = np.sum(y**2)
    
    return X_KTX_K, X_KTy, y_normsq, X_K_scaled
def solve_cvx(X_KTX_K, X_KTy, y_normsq, ff, beta, SCS_max_iters, loss_name, X_V=None, y=None):
    # poly act scalar output
    Z1 = cp.Variable((ff, ff), symmetric=True)
    Z2 = cp.Variable((ff, 1))
    Z4 = cp.Variable((1,1))

    Z1_prime = cp.Variable((ff, ff), symmetric=True)
    Z2_prime = cp.Variable((ff,1))
    Z4_prime = cp.Variable((1,1))


    zz = cp.vstack((cp.reshape((Z1-Z1_prime), (ff**2,1)), (Z2-Z2_prime), (Z4-Z4_prime)))
    
    
    yhat = X_V@zz
    if loss_name == "squared_loss":
        objective = 0.5*cp.sum_squares(yhat - y) + beta*(Z4 + Z4_prime)
    elif loss_name == "l1_loss":
        objective = cp.sum(cp.abs(yhat - y)) + beta*(Z4 + Z4_prime)
    elif loss_name == "huber":
        objective = cp.sum(cp.huber(yhat-y)) + beta*(Z4 + Z4_prime)


    Z = cp.vstack((cp.hstack((Z1, Z2)), cp.hstack((Z2.T, Z4))))
    Z_prime = cp.vstack((cp.hstack((Z1_prime, Z2_prime)), cp.hstack((Z2_prime.T, Z4_prime))))
    
    constraints = [Z >> 0] + [Z_prime >> 0]

    constraints += [cp.trace(Z1) == Z4]
    constraints += [cp.trace(Z1_prime) == Z4_prime]
    
    prob = cp.Problem(cp.Minimize(objective), constraints)
    start_time = time.time()
    print("started..")
    prob.solve(warm_start=False, max_iters=SCS_max_iters)
    end_time = time.time()
    print("time elapsed: " + str(end_time - start_time))

    # Print result.
    print(prob.status)
    print("The optimal value is", prob.value)
    print("The optimal value is", objective.value)
    
    return Z, Z_prime, objective.value

# neural decomposition function
def neural_decomposition(Z_decomp, tolerance=10**(-9)):
    # decomposes Z_decomp as a sum of r (where r=rank(Z_decomp)) rank-1 matrices \sum_{j=1}^r y_jy_j^T where
    # y_j^TGy_j = 0 for all j=1,...,r
    # based on the alg given in the proof of lemma 2.4 of the paper 'A Survey of the S-Lemma'
    G = np.identity(Z_decomp.shape[0])
    G[-1,-1] = -1

    # step 0
    evals, evecs = np.linalg.eigh(Z_decomp)
    # some eigvals are negative due to numerical issues, tolerance masking deals with that
    ind_pos_evals = (evals > tolerance)
    p_i_all = evecs[:,ind_pos_evals] * np.sqrt(evals[ind_pos_evals])

    outputs_y = np.zeros(p_i_all.shape)

    for i in range(outputs_y.shape[1]-1):
        # step 1
        p_1 = p_i_all[:,0:1]
        p_1Gp_1 = np.matmul(p_1.T, np.matmul(G, p_1))

        if p_1Gp_1 == 0:
            y = p_1.copy()

            # update
            p_i_all = np.delete(p_i_all, 0, 1) # delete the first column
        else:
            for j in range(1, p_i_all.shape[1]):
                p_j = p_i_all[:,j:j+1]
                p_jGp_j = np.matmul(p_j.T, np.matmul(G, p_j))
                if p_1Gp_1 * p_jGp_j < 0:
                    break

            # step 2
            p_1Gp_j = np.matmul(p_1.T, np.matmul(G, p_j))
            discriminant = 4*p_1Gp_j**2 - 4*p_1Gp_1*p_jGp_j
            alpha = (-2*p_1Gp_j + np.sqrt(discriminant)) / (2*p_jGp_j)
            y = (p_1 + alpha*p_j) / np.sqrt(1+alpha**2)

            # update
            p_i_all = np.delete(p_i_all, j, 1) # delete the jth column
            p_i_all = np.delete(p_i_all, 0, 1) # delete the first column

            u = (p_j - alpha*p_1) / np.sqrt(1+alpha**2)
            p_i_all = np.concatenate((p_i_all, u), axis=1) # insert u to the list of p_i's

        # save y
        outputs_y[:,i:i+1] = y.copy()

    # save the remaining column
    outputs_y[:, -1:] = p_i_all.copy()
    
    return outputs_y

# forward prop
def forward_prop_polyact(X, first_layer_weights, second_layer_weights, a, b, c):
    Xu = np.matmul(X, first_layer_weights)
    Xu_act = a*(Xu)**2 + b*(Xu) + c
    output = np.matmul(Xu_act, second_layer_weights)
    return output

### Parameters

In [ ]:
# parameter selection
a = 0.09 # polynomial activation coefficients: a, b, c
b = 0.5
c = 0.47
beta = 10**(-1) # regularization parameter
verbose = True
scs_max_iters = 50000 # maximum number of iterations for the convex solver
tol = 10**(-6) # tolerance parameter

loss_type = "squared_loss" # pick from: "squared_loss", "huber", "l1_loss" (other convex losses can be implemented)

### Generate random data

In [ ]:
# generate random data 
n, d = 100, 20
m_pnt = 5 # the number of planted neurons

np.random.seed(0)
A = np.random.normal(0,1,(n,d))**4

u_truth = np.random.normal(0,1,(d,m_pnt))
alpha_truth = np.random.normal(0,1,(m_pnt,))
noise = 1 * np.random.normal(0,0.1,(n,1))
y = np.sum((a*np.matmul(A, u_truth)**2 + b*np.matmul(A, u_truth) + c) * alpha_truth, axis=1, keepdims=True) + noise

A_test = A.copy()
y_test = y.copy()

print("A.shape = {}, y.shape = {}".format(A.shape, y.shape))
print("num of planted neurons = {}".format(m_pnt))

### Solver

In [ ]:
start_time = time.time()
# call to the main solver function
Z, Z_prime, noncvx_cost = convexNN_poly_act_solver(A, y, beta, scs_max_iters, loss_type, a, b, c)
end_time = time.time()

time_elapsed_cvx = end_time - start_time
print("total time: " + str(time_elapsed_cvx))

num_neurons_cvx = np.sum(np.linalg.eig(Z.value)[0] > tol) + np.sum(np.linalg.eig(Z_prime.value)[0] > tol)
print("The number of neurons is", num_neurons_cvx)

In [ ]:
# extract the neural network weights from the cvxpy solution
tolerance = tol # if the tolerance is too low, 'special_decomposition' might throw an error (NaN error), 
# in that case choosing a larger tolerance might help
decomp = neural_decomposition(Z.value, tolerance)
decomp_prime = neural_decomposition(Z_prime.value, tolerance)

first_layer_weights = np.concatenate((decomp[:-1, :], decomp_prime[:-1, :]), axis=1)
first_layer_weights = first_layer_weights / np.sqrt(np.sum(first_layer_weights**2, axis=0))
signs_second_layer = np.sign(np.concatenate((decomp[-1:, :], decomp_prime[-1:, :]), axis=1).T)
first_layer_weights = first_layer_weights * signs_second_layer[:,0]
second_layer_weights = np.concatenate((decomp[-1:, :]**2, -decomp_prime[-1:, :]**2), axis=1).T

print("shape of first layer weights: ", first_layer_weights.shape)
print("shape of second layer weights: ", second_layer_weights.shape)

### Vector output networks

In [ ]:
def cvx_polyact_vector_noquadform(A, y_onehot, beta, scs_max_iters, a, b, c):
    # same as cvx_polyact_vector except Y_hat is computed first and then objective is formed using frobenius norm
    Z1 = []; Z2 = []; Z4 = []; Z1_prime = []; Z2_prime = []; Z4_prime = []
    C = y_onehot.shape[1]
    n, d = A.shape
    
    for k in range(C):
        Z1.append(cp.Variable((d,d), symmetric=True))
        Z2.append(cp.Variable((d,1)))
        Z4.append(cp.Variable((1,1)))

        Z1_prime.append(cp.Variable((d,d), symmetric=True))
        Z2_prime.append(cp.Variable((d,1)))
        Z4_prime.append(cp.Variable((1,1)))

        
    Yhat = cp.Variable((n,C))
    constraints = []
    
    for j in range(n):
        xj = A[j:j+1,:].T
        for k in range(C):
            constraints = constraints + [Yhat[j,k] == a*(xj.T@(Z1[k]-Z1_prime[k])@xj) + b*(xj.T@(Z2[k]-Z2_prime[k])) + c*(Z4[k]-Z4_prime[k])]
    objective = 0.5 * cp.sum_squares(y_onehot-Yhat)

    for k in range(C):
        objective = objective + (beta*(Z4[k] + Z4_prime[k]))

    Z = []; Z_prime = []
    for k in range(C):
        Z.append(cp.vstack((cp.hstack((Z1[k], Z2[k])), cp.hstack((Z2[k].T, Z4[k])))))
        Z_prime.append(cp.vstack((cp.hstack((Z1_prime[k], Z2_prime[k])), cp.hstack((Z2_prime[k].T, Z4_prime[k])))))

        constraints = constraints + [Z[k] >> 0]
        constraints = constraints + [Z_prime[k] >> 0]

        constraints = constraints + [cp.trace(Z1[k]) == Z4[k]]
        constraints = constraints + [cp.trace(Z1_prime[k]) == Z4_prime[k]]


    prob = cp.Problem(cp.Minimize(objective), constraints)

    start_time = time.time()
    prob.solve(warm_start=False, max_iters=scs_max_iters)
    end_time = time.time()
    print("time elapsed: " + str(end_time - start_time))

    # Print result.
    print("prob.status = ", prob.status)
    print("The optimal value is", prob.value)
    print("The optimal value is", objective.value)

    return Z, Z_prime, end_time-start_time, objective.value

In [ ]:
# UCI datasets
uci_dataset_name = "statlog_vehicle_multiclass"
A = np.load("A_{}_train.npy".format(uci_dataset_name))
y = np.load("y_{}_train.npy".format(uci_dataset_name))
A_test = np.load("A_{}_test.npy".format(uci_dataset_name))
y_test = np.load("y_{}_test.npy".format(uci_dataset_name))


C = y.max()+1 # output dimension (number of classes)

y_onehot = np.zeros((y.size, C))
y_onehot[np.arange(y.size), y[:,0]] = 1

y_test_onehot = np.zeros((y_test.size, C))
y_test_onehot[np.arange(y_test.size), y_test[:,0]] = 1

n, d = A.shape
print(A.shape, y_onehot.shape, A_test.shape, y_test_onehot.shape)

In [ ]:
# parameter selection
a = 0.09 # polynomial activation coefficients: a and b
b = 0.5
c = 0.47
beta = 10**(0) # regularization parameter
verbose = True
scs_max_iters = 50000
tol = 10**(-6)

In [ ]:
Z, Z_prime, time_elapsed_cvx, noncvx_cost = cvx_polyact_vector_noquadform(A, y_onehot, beta, scs_max_iters, a, b, c)
noncvx_cost = noncvx_cost[0][0]

num_neurons_cvx = 0
for k in range(len(Z)):
    num_neurons_cvx += np.sum(np.linalg.eig(Z[k].value)[0] > tol)
    num_neurons_cvx += np.sum(np.linalg.eig(Z_prime[k].value)[0] > tol)
print("The number of neurons is", num_neurons_cvx)

In [ ]:
# compute test loss and accuracies
y_hat = np.zeros(y_onehot.shape)
for j in range(n):
    xj = A[j:j+1,:].T
    for k in range(C):
        quad_term = a * (xj.T @ (Z[k][0:d,0:d]-Z_prime[k][0:d,0:d]) @ xj)
        lin_term = b * (xj.T @ (Z[k][:d,d:]-Z_prime[k][:d,d:]))
        cons_term = c * (Z[k][d,d]-Z_prime[k][d,d])
        y_hat[j,k] = quad_term.value + lin_term.value + cons_term.value
        
y_hat_test = np.zeros(y_test_onehot.shape)
for j in range(A_test.shape[0]):
    xj = A_test[j:j+1,:].T
    for k in range(C):
        quad_term = a * (xj.T @ (Z[k][0:d,0:d]-Z_prime[k][0:d,0:d]) @ xj)
        lin_term = b * (xj.T @ (Z[k][:d,d:]-Z_prime[k][:d,d:]))
        cons_term = c * (Z[k][d,d]-Z_prime[k][d,d])
        y_hat_test[j,k] = quad_term.value + lin_term.value + cons_term.value

reg_term = 0
for k in range(C):
    reg_term = reg_term + beta*(Z[k][d,d]+Z_prime[k][d,d]).value

noncvx_cost = 0.5 * np.sum((y_onehot - y_hat)**2) + reg_term
noncvx_cost_test = 0.5 * np.sum((y_test_onehot - y_hat_test)**2) + reg_term


training_acc = np.sum(y[:,0] == np.argmax(y_hat, axis=1)) / y.shape[0]
test_acc = np.sum(y_test[:,0] == np.argmax(y_hat_test, axis=1)) / y_test.shape[0]

print("costs:", noncvx_cost, noncvx_cost_test)
print("accuracies:", training_acc, test_acc)